In [1]:
from datasets import load_dataset

dataset = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation")
dataset

Reusing dataset librispeech_asr (C:\Users\ayber\.cache\huggingface\datasets\hf-internal-testing___librispeech_asr\clean\2.1.0\f2c70a4d03ab4410954901bde48c54b85ca1b7f9bf7d616e7e2a72b5ee6ddbfc)


Dataset({
    features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
    num_rows: 73
})

In [2]:
audio_sample = dataset[2]
audio_sample["text"].lower()

'he tells us that at this festive season of the year with christmas and roast beef looming before us similes drawn from eating and its results occur most readily to the mind'

In [5]:
from transformers import Wav2Vec2Processor, TFWav2Vec2ForCTC

processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = TFWav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

Downloading:   0%|          | 0.00/159 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/163 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/291 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/360M [00:00<?, ?B/s]

Some layers from the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing TFWav2Vec2ForCTC: ['dropout_50']
- This IS expected if you are initializing TFWav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFWav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFWav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['dropout_101']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
inputs = processor(audio_sample["audio"]["array"], sampling_rate=audio_sample["audio"]["sampling_rate"], padding="longest")

In [13]:
from datasets import load_dataset

# load dummy dataset and read soundfiles
ds = load_dataset("patrickvonplaten/librispeech_asr_dummy", "clean", split="validation")

Downloading:   0%|          | 0.00/5.16k [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

0 examples [00:00, ? examples/s]

Dataset librispeech_asr downloaded and prepared to C:\Users\ayber\.cache\huggingface\datasets\patrickvonplaten___librispeech_asr\clean\2.1.0\f2c70a4d03ab4410954901bde48c54b85ca1b7f9bf7d616e7e2a72b5ee6ddbfc. Subsequent calls will reuse this data.


In [15]:
# tokenize
input_values = processor(ds[0]["audio"]["array"], sampling_rate=audio_sample["audio"]["sampling_rate"], padding="longest").input_values  # Batch size 1

In [17]:
import tensorflow as tf

logits = model(input_values).logits

ValueError: Exception encountered when calling layer "conv_layers.0" (type TFWav2Vec2GroupNormConvLayer).

Input 0 of layer "conv" is incompatible with the layer: expected min_ndim=3, found ndim=2. Full shape received: (93680, 1)

Call arguments received:
  • hidden_states=tf.Tensor(shape=(93680, 1), dtype=float32)

In [ ]:
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)

transcription[0].lower()